In [1]:
from mycnn import AlexNet
import tensorflow as tf

In [2]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r'D:\Datasets\DogsVsCats\train',
    image_size=(227,227),
    batch_size=50,
    label_mode="categorical",
    validation_split=0.2,
    subset="training",
    seed=10
)
train_dataset.map(lambda x, y: (x/255., y))


valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r'D:\Datasets\DogsVsCats\train',
    image_size=(227,227),
    batch_size=50,
    label_mode="categorical",
    validation_split=0.2,
    subset="validation",
    seed=10
)
valid_dataset.map(lambda x, y: (x/255., y))


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


<MapDataset shapes: ((None, 227, 227, 3), (None, 2)), types: (tf.float32, tf.float32)>

In [3]:
alexnet = AlexNet(classes_num=2)
alexnet.summary()

Model: "AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
re_lu (ReLU)                 (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024

In [4]:
alexnet.setup_training(
    'log_alexnet',
    epochs=10, 
    batch_size=50,  # batch size depend on ImageGenerator
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
alexnet.add_callback(
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.8, patience=10,
        min_lr=0.00001,
        verbose=1
    )
)

[Info] Create new the directory for training log !!!!!
******************************
Training new model...
******************************
checkpoint_path : log_alexnet\weights
best_checkpoint : log_alexnet\weights.h5
epochs          : 50
initial_epoch   : 0
batch_size      : 50
optimizer       : <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x00000191E79E54C8>
loss            : categorical_crossentropy
metrics         : ['accuracy']


In [5]:
alexnet.train_generator(train_dataset, valid_dataset)

Epoch 1/50
400/400 [==============================] - 99s 236ms/step - loss: 1.6152 - accuracy: 0.5214 - val_loss: 0.6717 - val_accuracy: 0.5730

Epoch 00001: val_loss improved from inf to 0.67174, saving model to log_alexnet\weights.h5
Epoch 2/50
400/400 [==============================] - 105s 261ms/step - loss: 0.6703 - accuracy: 0.5883 - val_loss: 0.6433 - val_accuracy: 0.6368

Epoch 00002: val_loss improved from 0.67174 to 0.64333, saving model to log_alexnet\weights.h5
Epoch 3/50
400/400 [==============================] - 115s 286ms/step - loss: 0.6416 - accuracy: 0.6379 - val_loss: 0.6361 - val_accuracy: 0.6226

Epoch 00003: val_loss improved from 0.64333 to 0.63614, saving model to log_alexnet\weights.h5
Epoch 4/50
400/400 [==============================] - 117s 289ms/step - loss: 0.5896 - accuracy: 0.6880 - val_loss: 0.5733 - val_accuracy: 0.6944

Epoch 00004: val_loss improved from 0.63614 to 0.57331, saving model to log_alexnet\weights.h5
Epoch 5/50
400/400 [=================

KeyboardInterrupt: 